# Demo of SQL DB Interface Using db_CONN Class 

## Requirements
- Custom library: microsrv_interface.comm_interface
    - Classes: db_CONN, set_env_param
- pysftp==0.2.9
- pika==1.1.0
- path==14.0.1
- pyodbc==4.0.30
- pandas==1.2.1

In [9]:
import os
import sys
import traceback
import time
import pandas as pd
import json

In [10]:
# Custom handeler libray 
# Maintained by Darryl Bryson
# Repo: https://github.com/DarrylBrysonDev0/ms-interface-lib.git

from microsrv_interface.comm_interface import db_CONN, set_env_param

Load sample data from csv to dataframe

In [11]:
sample_data_df = pd.read_csv('./data/sample_lab_measurement.csv') 
sample_data_df = sample_data_df.replace('\'', '',regex=True)
sample_data_df.head()

,machine_id,test_id,technician,test_routine,batched,loc_1_x_offset,loc_1_y_offset,loc_1_z_offset,loc_2_x_offset,loc_2_y_offset,loc_2_z_offset
0,Machine_13,15ce86ab99b3490c96ee855c8347e66b,Jennifer Green,"[d, f, c, b, e]",NaN,14.27,0.295528,3072,10.12,0.953938,4681
1,Machine_23,efabe31a552649698f04c5e251a88112,Stephanie Collins,"[a, c, d, e, f, b]",No,-4.78,0.579363,4453,0.66,0.653040,4747
2,Machine_01,876c727a38274fe59353a76289553a6a,Ryan Page,"[f, a, d, b, e, c]",NaN,13.80,0.961564,4824,-7.30,0.842194,4724
3,Machine_15,84f2c6cb19a04fa0ad9dd135ca10b7af,Willie Golden,"[a, c, e, d]",Yes,3.42,0.838260,4970,3.75,0.466604,2875
4,Machine_20,76faffd74aab4d189876df0249fb3a87,Emily Allen,"[b, f]",Yes,5.84,0.499678,2782,-12.86,0.919360,3112


## Set connection parameters

In [12]:
# Target db table of load
db_tbl_name = '[telem-dev-db].[dbo].[sample_lab_measurement]'

In [13]:
# MS-SQL db connection details
res_dic = {}
res_dic['ServerAddr']='192.168.86.33'
res_dic['DBName']='telem-dev-db'
res_dic['UserName']='sa'
res_dic['Password']='Testing1122'

db_configData = json.dumps(res_dic)

In [14]:
bufferSize = 1000

## db_CONN Utilization
db_CONN Class default configures to environment variables (for docker use cases)
To compensate we reconfigure the instance:
```python
db_interface.configData = db_connStr
db_interface.ingest_buffer_size = bufferSize
db_interface.IS_VERBOSE = True
```
**Parameters**
- configData => MS SQL connection string as JSON object
- ingest_buffer_size => Insert buffer size (Greatly effects overall load time)
- IS_VERBOSE => Get process details

In [15]:
def main(dataDF, db_connStr, targetTBL, bufferSize):
    # Set Database interface
    db_interface = db_CONN()

    try:
        # Connect to Database
        print(' [*] Connecting to database')
        with db_interface as db_conn:
            print(' [+] Connected to database')

            # Load dataframe
            df = dataDF
            
            # Reconfigure connection parameters
            # Default db_CONN configures 
            db_interface.configData = db_connStr
            db_interface.ingest_buffer_size = bufferSize
            db_interface.IS_VERBOSE = True
            
            # Set dataframe property
            db_interface.set_df(df,targetTBL)

            # Write dataframe to linked database table
            row_cnt = db_interface.insert_dataframe()
            print(' [*] Total records writen: {0}'.format(row_cnt))
            print()
            
    except Exception as err:
        print()
        print("An error occurred while executing main proc.")
        print(str(err))
        traceback.print_tb(err.__traceback__)
    return

In [16]:
# Execute main method
main(sample_data_df, db_configData, db_tbl_name, bufferSize)

 [*] Connecting to database
 [+] Connected to database
 [*] Starting db write
Connection established
 [+] Buffer reached
 [I] Row index: 1000
 [+] Buffer reached
 [I] Row index: 2000
 [+] Buffer reached
 [I] Row index: 3000
 [+] Buffer reached
 [I] Row index: 4000
 [+] Buffer reached
 [I] Row index: 5000
 [+] Buffer reached
 [I] Row index: 6000
 [+] Buffer reached
 [I] Row index: 7000
 [+] Buffer reached
 [I] Row index: 8000
 [+] Buffer reached
 [I] Row index: 9000
 [+] Buffer reached
 [*] EOD reached
 [I] Row index: 10000
Results written in: 2.200498580932617
 [*] Total records writen: 10000

